In [47]:
import pandas as pd
import numpy as np
from sklearn import metrics


In [3]:
data_df = pd.read_pickle('dev_test.pkl')

In [4]:
data_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,main_text_kg,Unnamed: 0,claim_kg_size,main_text_kg_size,clean_claim_triples,clean_main_text_triples,claim_evidence_1,entailment_scores_1
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,"[{'head': 'Confederate flag', 'type': 'instanc...",NaN,3,14,"[(A baby died at an unnamed medical facility, ...","[(Confederate flag, instance of, racist), (Fie...","{'(A baby died at an unnamed medical facility,...","[[0.66573095, -0.56622183, -0.4859991], [0.430..."
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...","[{'head': 'KSNT', 'type': 'located in the admi...",NaN,3,6,"[(Shawnee County, located in the administrativ...","[(KSNT, located in the administrative territor...","{'(Shawnee County, located in the administrati...","[[0.6284045, -0.64509016, -0.4347026], [0.6175..."
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...","[{'head': 'Express', 'type': 'country', 'tail'...",NaN,3,36,"[(banned pork from school canteens, country, G...","[(Express, country, British), (Express, instan...","{'(banned pork from school canteens, country, ...","[[0.64209604, -0.69231135, -0.32926825], [0.68..."
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...","[{'head': 'heroin', 'type': 'instance of', 'ta...",NaN,3,21,"[(Coronavirus, country, Canada), (safe drugs, ...","[(heroin, instance of, controlled substance), ...","{'(Coronavirus, country, Canada)': ['(coronavi...","[[-0.1503326, -0.75800896, 0.63468295], [0.660..."
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...","[{'head': 'oak forests', 'type': 'located in t...",NaN,3,3,"[(Wayne National Forest, instance of, wildlife...","[(oak forests, located in the administrative t...","{'(Wayne National Forest, instance of, wildlif...","[[0.63135314, -0.7029956, -0.32739952], [0.662..."


In [59]:
def get_verdict(row, low, high):
    entailment_scores = row['entailment_scores_1']
    # print(type(entailment_scores))
    index_values = [-1, 1, 0]
    verdict = 0

    for subclaim_entailment_scores in entailment_scores:
        # print(subclaim_entailment_scores)
        which_verdict = subclaim_entailment_scores.argmax()
        # print(which_verdict)
        verdict += index_values[which_verdict]*subclaim_entailment_scores[which_verdict]

    if verdict > high:
        return 1
    elif verdict > low:
        return 0
    else:
        return -1

In [60]:
highs = np.arange(-1, 1, 0.01)
lows = np.arange(-1, 1, 0.01)

best_f1_score = -1
best_high = 0.1
best_low = -0.1
for high in highs:
    for low in lows:
        if low < high:
            data_df['verdict'] = data_df.apply(lambda row: get_verdict(row, low, high), axis=1)

            f1 = metrics.f1_score(data_df['verdict_actual'], data_df['verdict'], labels=[-1, 1, 0], average='weighted')

            if f1 > best_f1_score:
                best_f1_score = f1
                best_low = low
                best_high = high

print(best_f1_score)
print(best_low)
print(best_high)

data_df['verdict'] = data_df.apply(lambda row: get_verdict(row, best_low, best_high), axis=1)

0.3429609948878881
-0.9199999999999999
-0.5699999999999996


In [26]:
data_df['verdict_actual'] = data_df['label'].apply(lambda label: 1 if label=='true' else (-1 if label=='false' else 0))

In [58]:
target_names = ['contradiction', 'support', 'nei']
labels = [-1, 1, 0]

print(metrics.classification_report(data_df['verdict_actual'], data_df['verdict'], target_names=target_names, labels=labels))
print(metrics.accuracy_score(data_df['verdict_actual'], data_df['verdict']))

               precision    recall  f1-score   support

contradiction       0.28      0.27      0.27       768
      support       0.47      0.59      0.52      1228
          nei       0.20      0.07      0.11       456

     accuracy                           0.39      2452
    macro avg       0.31      0.31      0.30      2452
 weighted avg       0.36      0.39      0.37      2452

0.39110929853181076


AxisError: axis 1 is out of bounds for array of dimension 1

In [53]:
data_df.to_pickle('dev_test_verdict.pkl')

In [54]:
data_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,main_text_kg,Unnamed: 0,claim_kg_size,main_text_kg_size,clean_claim_triples,clean_main_text_triples,claim_evidence_1,entailment_scores_1,verdict,verdict_actual
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,"[{'head': 'Confederate flag', 'type': 'instanc...",NaN,3,14,"[(A baby died at an unnamed medical facility, ...","[(Confederate flag, instance of, racist), (Fie...","{'(A baby died at an unnamed medical facility,...","[[0.66573095, -0.56622183, -0.4859991], [0.430...",-1,0
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...","[{'head': 'KSNT', 'type': 'located in the admi...",NaN,3,6,"[(Shawnee County, located in the administrativ...","[(KSNT, located in the administrative territor...","{'(Shawnee County, located in the administrati...","[[0.6284045, -0.64509016, -0.4347026], [0.6175...",-1,1
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...","[{'head': 'Express', 'type': 'country', 'tail'...",NaN,3,36,"[(banned pork from school canteens, country, G...","[(Express, country, British), (Express, instan...","{'(banned pork from school canteens, country, ...","[[0.64209604, -0.69231135, -0.32926825], [0.68...",-1,-1
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...","[{'head': 'heroin', 'type': 'instance of', 'ta...",NaN,3,21,"[(Coronavirus, country, Canada), (safe drugs, ...","[(heroin, instance of, controlled substance), ...","{'(Coronavirus, country, Canada)': ['(coronavi...","[[-0.1503326, -0.75800896, 0.63468295], [0.660...",-1,1
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...","[{'head': 'oak forests', 'type': 'located in t...",NaN,3,3,"[(Wayne National Forest, instance of, wildlife...","[(oak forests, located in the administrative t...","{'(Wayne National Forest, instance of, wildlif...","[[0.63135314, -0.7029956, -0.32739952], [0.662...",-1,1
